In [1]:
import pprint

import torch
from torch import MobileOptimizerType
import torch.optim as optim
import torch.nn as nn

from seq2seq.model.data_loader import DataLoader
import seq2seq.model.data_loader as data_loader
from seq2seq.model.seq2seq import Seq2Seq

from seq2seq.model.trainer import SingleTrainer
from seq2seq.model.trainer import MaximumLikelihoodEstimationEngine

In [2]:
class Args :
    def __init__(self):
        self.model_fn = './model.pth'
        self.gpu_id = 0
        self.n_epochs = 5
        self.lr_scheduler = None
        self.verbose = 2
        self.init_epoch = 0
        self.iteration_per_update = 1
        self.max_grad_norm = 5.
        self.use_noam_decay = False

In [3]:
config = Args()

def print_config(config):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(vars(config))

print_config(config)

{   'gpu_id': 0,
    'init_epoch': 0,
    'iteration_per_update': 1,
    'lr_scheduler': None,
    'max_grad_norm': 5.0,
    'model_fn': './model.pth',
    'n_epochs': 5,
    'use_noam_decay': False,
    'verbose': 2}


In [4]:
loader = DataLoader(
    './kor_eng_translation/train copy.tsv',
    './kor_eng_translation/valid copy.tsv',
    batch_size=20,
)
input_size, output_size = len(loader.src.vocab), len(loader.tgt.vocab)
print(input_size, output_size)

108264 120302


In [5]:
def get_crit(output_size, pad_index):
    loss_weight = torch.ones(output_size)
    loss_weight[pad_index] = 0.

    crit = nn.NLLLoss(weight=loss_weight, reduction='sum')
    return crit

model = Seq2Seq(input_size, 512, 512, output_size)
crit = get_crit(output_size, data_loader.PAD)

if config.gpu_id >= 0:
    model.cuda(config.gpu_id)
    crit.cuda(config.gpu_id)



optimizer = optim.Adam(model.parameters())

print(model)
print(crit)
print(optimizer)

Seq2Seq(
  (emb_src): Embedding(108264, 512)
  (emb_dec): Embedding(120302, 512)
  (encoder): Encoder(
    (rnn): LSTM(512, 256, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (rnn): LSTM(1024, 512, num_layers=4, batch_first=True, dropout=0.2)
  )
  (attention): Attention(
    (linear): Linear(in_features=512, out_features=512, bias=False)
    (softmax): Softmax(dim=-1)
  )
  (concat): Linear(in_features=1024, out_features=512, bias=True)
  (tanh): Tanh()
  (generator): Generator(
    (output): Linear(in_features=512, out_features=120302, bias=True)
    (softmax): LogSoftmax(dim=-1)
  )
)
NLLLoss()
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


In [6]:
trainer = SingleTrainer(MaximumLikelihoodEstimationEngine, config)
trainer.train(
    model,
    crit,
    optimizer,
    train_loader=loader.train_iter,
    valid_loader=loader.valid_iter,
    src_vocab=loader.src.vocab,
    tgt_vocab=loader.tgt.vocab,
    n_epochs=config.n_epochs,
    lr_scheduler=None,
)

  0%|                                                                                         | 1/18140 [00:00…

Current run is terminating due to exception: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 15.99 GiB total capacity; 12.74 GiB already allocated; 0 bytes free; 13.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
Engine run is terminating due to exception: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 15.99 GiB total capacity; 12.74 GiB already allocated; 0 bytes free; 13.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 15.99 GiB total capacity; 12.74 GiB already allocated; 0 bytes free; 13.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF